In [ ]:
from openai import OpenAI
import base64
import requests
import firebase_admin
from firebase_admin import credentials, storage
import uuid
import os

def encode_image_to_base64(path):
    with open(path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

if not firebase_admin._apps:
    cred = credentials.Certificate("/home/ddddewang/Desktop/jams-f92b1-firebase-adminsdk-fbsvc-8651bbf671.json")
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'jams-f92b1.firebasestorage.app'
    })
else:
    print("Here~!")
    app = firebase_admin.get_app()
    print(app.options.get('storageBucket', 'No storage bucket found'))
class ImageUploader:
    def __init__(self):
        pass
    def upload_to_firebase(self, image_path, myuuid):
        bucket = storage.bucket()
        blob = bucket.blob(f"uploads/{myuuid}/{uuid.uuid4()}{os.path.splitext(image_path)[1]}")
        blob.upload_from_filename(image_path)
        blob.make_public()
        return blob.public_url

class LLMChatBot:
    def __init__(self, model_name="gpt-4o", temperature=0.4, key=None, uuid=None):
        self.img_uploader = ImageUploader()
        self.model_name = model_name
        self.temperature = temperature
        self.messages = []
        assert key is not None, "API key must be provided"
        assert uuid is not None, "UUID must be provided"
        self.client = OpenAI(
            api_key=key
        )
        self.uuid = uuid
        print("LLMChatBot is initialized with model:", self.model_name)

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def add_message_front(self, role, content):
        if len(self.messages) == 0:
            self.messages.append({"role": role, "content": content})
        else:
            self.messages.insert(0, {"role": role, "content": content})
    
    def pop_message(self):
        if self.messages:
            return self.messages.pop()
        return None
    
    def pop_message_front(self):
        if self.messages:
            return self.messages.pop(0)
        return None

    def have_system_prompt(self):
        if len(self.messages)>0:
            return self.messages[0]['role'] == 'system'
        return False
    
    def clear_messages(self):
        self.messages = []

    def get_response(self, query, store=True):
        if store:
            self.add_message("user",query)
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=self.messages,
            temperature=self.temperature,
        )
        if store:
            self.add_message("assistant", response.choices[0].message.content)
        return response.choices[0].message.content
    
    def get_response_vision(self, query, image_path, text_store=True, image_store=False):
        image_url = self.img_uploader.upload_to_firebase(image_path, self.uuid)
        # image_b64 = encode_image_to_base64(image_path)
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=self.messages[0:] + [{
                    "role": "user", 
                    "content": [
                        {"type": "text", "text": query},
                        {"type": "image_url", "image_url": {"url": image_url}},
                    ], 
                }], 
            temperature=self.temperature,
        )
        if image_store:
            self.add_message("user",[
                        {"type": "text", "text": query},
                        {"type": "image_url", "image_url": {"url": image_url}},
                    ])
            self.add_message("assistant", response.choices[0].message.content)
        elif text_store:
            self.add_message("user", query)
            self.add_message("assistant", response.choices[0].message.content)

        return response.choices[0].message.content

class ChefBot(LLMChatBot):
    def __init__(self, model_name="gpt-4o", temperature=0.4, key=None, uuid=None, system_prompt_path="ChefBotSystemPrompt.txt"):
        super().__init__(model_name, temperature, key, uuid)
        self.init_system_prompt(system_prompt_path)

    def init_system_prompt(self, system_prompt_path):
        if self.have_system_prompt():
            self.pop_message_front()
        self.add_message_front("system",open(system_prompt_path, "r").read())

    def get_response_realtime_vision(self, query, image_path="vision.png", text_store=True, image_store=False):
        """
        The parameters:
            - image_path: needed to be updated with the precise image name.
        """

        return self.get_response_vision(query, image_path, text_store, image_store)

Here~!
jams-f92b1.firebasestorage.app


In [14]:
apikey = open("/home/ddddewang/Desktop/gptapi.txt", "r").read().strip()
myuuid = open("/home/ddddewang/Desktop/myuuid.txt", "r").read().strip()
MyBot = ChefBot(model_name="gpt-4o", temperature=1.0, key=apikey, uuid=myuuid)

LLMChatBot is initialized with model: gpt-4o


In [11]:
print(MyBot.get_response_vision("너의 시스템 프롬프트상 요리 레시피는 어떤 포맷으로 출력되어야 하지? 그 플랫폼에 맞춰서 내가 보여준 냉장고 이미지 내부의 재료들로 만들 수 있는 요리 레시피를 알려줘. 참고로 너는 레시피를 json 형태로만 출력해야해.", "refrigerator_example.jpg"))

SDK에서 참조하는 실제 버킷 이름: jams-f92b1.firebasestorage.app
죄송하지만 이미지를 분석하여 구체적인 재료를 식별할 수는 없습니다. 그러나 일반적인 예를 들어 JSON 형식으로 요리 레시피를 제공할 수 있습니다. 냉장고에 있는 일반적인 재료를 사용한 예시 JSON입니다:

```json
{
  "recipe": {
    "name": "Simple Salad",
    "ingredients": [
      "lettuce",
      "tomato",
      "cucumber",
      "cheese",
      "boiled egg",
      "olive oil",
      "balsamic vinegar"
    ],
    "instructions": [
      "Wash all vegetables thoroughly.",
      "Slice the tomato and cucumber.",
      "Dice the boiled egg.",
      "Combine all ingredients in a large bowl.",
      "Add sliced cheese on top.",
      "Drizzle with olive oil and balsamic vinegar, then mix well."
    ]
  }
}
```

이 형식을 따라 활용 가능한 다른 재료로 JSON 레시피를 만들어보세요!


In [15]:
print(MyBot.get_response("알리오 올리오 레시피 알려줘."))

{
  "name": "알리오 올리오",
  "ingredients": "스파게티 면 200g, 올리브 오일 100ml, 마늘 6쪽, 페페론치노 2개, 소금 약간, 파슬리 약간",
  "recipe": [
    "1. 마늘은 얇게 슬라이스합니다.",
    "2. 페페론치노는 반으로 쪼개거나 잘게 부숩니다.",
    "3. 냄비에 물을 끓이고 소금을 넣은 뒤, 스파게티 면을 포장지에 적힌 시간보다 1분 짧게 삶습니다.",
    "4. 면을 건져내어 물기를 빼고, 삶은 물을 한 컵 따로 보관합니다.",
    "5. 팬에 올리브 오일을 두르고 중약불에서 마늘을 볶아 향을 냅니다.",
    "6. 마늘이 약간 노릇해지면 페페론치노를 추가해 함께 볶습니다.",
    "7. 삶은 스파게티 면을 팬에 넣고, 준비한 면수를 조금씩 추가하며 섞어줍니다.",
    "8. 소금으로 간을 맞추고, 마지막으로 파슬리를 뿌려 섞습니다.",
    "9. 완성된 알리오 올리오를 접시에 담아냅니다."
  ]
}


In [46]:
from firebase_admin import storage
bucket = storage.bucket()
print("버킷 이름:", bucket.name)

버킷 이름: jams-f92b1.appspot.com
